In [85]:
import ee 
import geemap
import colorcet as cc

In [86]:
ee.Authenticate()

True

In [87]:
ee.Initialize(project='hotspotstoplight')

In [117]:
aoi = ee.Geometry.Polygon( # all of costa rica
    [[
        [-85.9, 8.0],  # Lower left corner (southwest)
        [-85.9, 11.2], # Upper left corner (northwest)
        [-82.5, 11.2], # Upper right corner (northeast)
        [-82.5, 8.0],  # Lower right corner (southeast)
        [-85.9, 8.0]   # Closing the polygon by repeating the first point
    ]]
)

scale = 90

In [118]:
# Applies scaling factors.
def apply_scale_factors(image):
    # Scale and offset values for optical bands
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    
    # Scale and offset values for thermal bands
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    
    # Add scaled bands to the original image
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

# Function to Mask Clouds and Cloud Shadows in Landsat 8 Imagery
def cloud_mask(image):
    # Define cloud shadow and cloud bitmasks (Bits 3 and 5)
    cloud_shadow_bitmask = 1 << 3
    cloud_bitmask = 1 << 5
    
    # Select the Quality Assessment (QA) band for pixel quality information
    qa = image.select('QA_PIXEL')
    
    # Create a binary mask to identify clear conditions (both cloud and cloud shadow bits set to 0)
    mask = qa.bitwiseAnd(cloud_shadow_bitmask).eq(0).And(qa.bitwiseAnd(cloud_bitmask).eq(0))
    
    # Update the original image, masking out cloud and cloud shadow-affected pixels
    return image.updateMask(mask)

In [135]:
startDate = '2023-01-01'
endDate = '2023-12-31'

scale = 90

In [136]:
# Import and preprocess Landsat 8 imagery
image = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
            .filterBounds(aoi) \
            .filterDate(startDate, endDate) \
            .map(apply_scale_factors) \
            .map(cloud_mask) \
            .median() \
            .clip(aoi)

# Calculate Normalized Difference Vegetation Index (NDVI)
ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

# Calculate the minimum and maximum NDVI value within the AOI
# Calculate the minimum and maximum NDVI value within the AOI with adjusted maxPixels and scale
ndvi_min = ee.Number(ndvi.reduceRegion(
    reducer=ee.Reducer.min(), 
    geometry=aoi, 
    scale=scale, 
    maxPixels=1e13  # Increase maxPixels here
).values().get(0))

ndvi_max = ee.Number(ndvi.reduceRegion(
    reducer=ee.Reducer.max(), 
    geometry=aoi, 
    scale=scale, 
    maxPixels=1e13  # Increase maxPixels here
).values().get(0))

# Fraction of Vegetation (FV) Calculation
fv = ndvi.subtract(ndvi_min).divide(ndvi_max.subtract(ndvi_min)).pow(2).rename('FV')

# Emissivity Calculation
em = fv.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename('EM')

ndbi = image.normalizedDifference(['SR_B6', 'SR_B5']).rename('NDBI')
ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')

In [137]:
# Select Thermal Band (Band 10) and Rename It
thermal = image.select('ST_B10').rename('thermal')

# Land Surface Temperature (LST) Calculation
lst = thermal.expression(
    '(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15', {
        'TB': thermal.select('thermal'), # Select the thermal band
        'em': em # Assign emissivity
    }).rename('LST')

In [138]:
landcover = ee.Image("ESA/WorldCover/v100/2020").select('Map').clip(aoi)

dem = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM").mosaic().clip(aoi)

In [139]:
image_for_sampling = ndvi \
    .addBands(em) \
    .addBands(ndbi) \
    .addBands(ndwi) \
    .addBands(landcover.rename('landcover')) \
    .addBands(dem.rename('elevation')) \
    .addBands(ee.Image.pixelLonLat()) \
    .addBands(lst) 

In [143]:
# Sample the combined image to create a feature collection for training
training_sample = image_for_sampling.sample(**{
    'region': aoi,
    'scale': scale,
    'numPixels': 25000,
    'seed': 0,
    'geometries': True  # Include geometries if needed for visualization
})

# Split the data into training and testing
training_sample = training_sample.randomColumn()
training = training_sample.filter(ee.Filter.lt('random', 0.7))
testing = training_sample.filter(ee.Filter.gte('random', 0.7))

# Train the Random Forest regression model
inputProperties=['NDVI', 'NDBI', 'NDWI', 'EM', 'longitude', 'latitude', 'landcover', 'elevation']
numTrees = 10  # Number of trees in the Random Forest
regressor = ee.Classifier.smileRandomForest(numTrees).setOutputMode('REGRESSION').train(
    training, 
    classProperty='LST', 
    inputProperties=inputProperties
)

# Apply the trained model to the image
predicted_image = image_for_sampling.select(inputProperties).classify(regressor)

difference = lst.subtract(predicted_image).rename('difference')

In [144]:
# Assuming 'max_lst' is your actual maximum LST image and 'predicted_image' contains the predictions
# Calculate the squared difference between actual and predicted LST
squared_difference = lst.subtract(predicted_image).pow(2).rename('difference')

# Reduce the squared differences to get the mean squared difference over your area of interest (aoi)
mean_squared_error = squared_difference.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=aoi,
    scale=scale,  # Adjust scale to match your dataset's resolution
    maxPixels=1e14
)

# Calculate the square root of the mean squared error to get the RMSE
rmse = mean_squared_error.getInfo()['difference'] ** 0.5

print('RMSE:', rmse)

RMSE: 2.4949984628837494


In [146]:
vizParams = {
    'min': 0,
    'max': 45,
    'palette': cc.fire
}

m = geemap.Map()
m.centerObject(aoi, 8)
m.add("basemap_selector")
m.add("layer_manager")
m.addLayer(lst, vizParams, 'Actual Max LST')
m.addLayer(predicted_image, vizParams, 'Predicted LST')
m.addLayer(difference, {'min': -10, 'max': 10, 'palette': cc.cwr}, 'Difference')
m

Map(center=[9.60162237566943, -84.19999999999996], controls=(WidgetControl(options=['position', 'transparent_b…